# Street microgrid

Initial part is some EDA and trying to clean data

In [1]:
# basic libraries
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from tqdm import tqdm, trange

In [2]:
dirpath = r'C:\Users\joeba\OneDrive\Documents\ML\smart meter'

In [26]:
# smart meter data
smdpath = dirpath+'/Power-Networks-LCL-June2015(withAcornGps).csv_Pieces'

smart_meter_data = pd.read_csv(smdpath+'/Power-Networks-LCL-June2015(withAcornGps)v2_1.csv', 
                               parse_dates=['DateTime'],
                               dayfirst=True,
                               usecols=['DateTime', 'KWH/hh (per half hour) '] )

smart_meter_data['KWH/hh (per half hour) '].fillna(0, inplace=True)
smart_meter_data['KWH/hh (per half hour) '].replace('Null', 0.0, inplace=True)
smart_meter_data['KWH/hh (per half hour) '] = smart_meter_data['KWH/hh (per half hour) '].astype(float)
smart_meter_data['house'] = 1

smart_meter_data['Date'] = smart_meter_data['DateTime'].dt.strftime('%Y-%m-%d')
smart_meter_data['Date'] = pd.to_datetime(smart_meter_data['Date'], dayfirst=True)

smart_meter_data = smart_meter_data.drop_duplicates(subset=['DateTime'], keep='first')

# reduce to one year
smart_meter_data = smart_meter_data[smart_meter_data['Date'] > '2012-12-31']
smart_meter_data = smart_meter_data[smart_meter_data['Date'] < '2014-01-01']

smart_meter_data.shape

smart_meter_data = smart_meter_data[['Date', 'DateTime', 'KWH/hh (per half hour) ']]
smart_meter_data.columns = ['Date', 'DateTime', 'h1']

In [28]:
smart_meter_data

,Date,DateTime,h1
3840,2013-01-01,2013-01-01 00:00:00,0.219
3841,2013-01-01,2013-01-01 00:30:00,0.241
3842,2013-01-01,2013-01-01 01:00:00,0.191
3843,2013-01-01,2013-01-01 01:30:00,0.235
3844,2013-01-01,2013-01-01 02:00:00,0.182
...,...,...,...
21367,2013-12-31,2013-12-31 21:30:00,0.451
21368,2013-12-31,2013-12-31 22:00:00,0.456
21369,2013-12-31,2013-12-31 22:30:00,0.402
21370,2013-12-31,2013-12-31 23:00:00,0.491


In [32]:
smart_meter_data.dtypes

Date        datetime64[ns]
DateTime    datetime64[ns]
h1                 float64
h2                 float64
h3                 float64
h4                 float64
h5                 float64
h6                 float64
h7                 float64
h8                 float64
h9                 float64
dtype: object

In [29]:

for i in trange(2, 10):

    h2 = pd.read_csv(smdpath+'/Power-Networks-LCL-June2015(withAcornGps)v2_'+str(i)+'.csv', 
                                   parse_dates=['DateTime'],
                                   dayfirst=True,
                                   usecols=['DateTime', 'KWH/hh (per half hour) '] )

    h2['KWH/hh (per half hour) '].fillna(0, inplace=True)
    h2['KWH/hh (per half hour) '].replace('Null', 0.0, inplace=True)
    h2['KWH/hh (per half hour) '] = h2['KWH/hh (per half hour) '].astype(float)
    h2['house'] = 1

    h2['Date'] = h2['DateTime'].dt.strftime('%Y-%m-%d')
    h2['Date'] = pd.to_datetime(h2['Date'], dayfirst=True)

    h2 = h2.drop_duplicates(subset=['DateTime'], keep='first')

    # reduce to one year
    h2 = h2[h2['Date'] > '2012-12-31']
    h2 = h2[h2['Date'] < '2014-01-01']

    h2 = h2[['DateTime', 'KWH/hh (per half hour) ']]
    h2.columns = ['DateTime', 'h'+str(i)]
    #h2.shape
    
    smart_meter_data = pd.merge(smart_meter_data, h2, how='left', on='DateTime')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:05<00:00,  8.24s/it]


In [30]:
smart_meter_data

,Date,DateTime,h1,h2,h3,h4,h5,h6,h7,h8,h9
0,2013-01-01,2013-01-01 00:00:00,0.219,0.006,0.093,0.073,0.107,0.235,0.032,0.076,0.152
1,2013-01-01,2013-01-01 00:30:00,0.241,0.017,0.068,0.059,0.103,0.201,0.034,0.075,0.111
2,2013-01-01,2013-01-01 01:00:00,0.191,0.059,0.078,0.069,0.103,0.202,0.013,0.073,0.032
3,2013-01-01,2013-01-01 01:30:00,0.235,0.000,0.228,0.059,0.130,0.162,0.043,0.073,0.068
4,2013-01-01,2013-01-01 02:00:00,0.182,0.000,0.097,0.088,0.078,0.175,0.069,0.072,0.037
...,...,...,...,...,...,...,...,...,...,...,...
17515,2013-12-31,2013-12-31 21:30:00,0.451,0.056,0.096,0.173,0.140,0.710,0.093,0.087,0.622
17516,2013-12-31,2013-12-31 22:00:00,0.456,0.017,0.107,0.166,0.158,0.265,0.104,0.095,0.553
17517,2013-12-31,2013-12-31 22:30:00,0.402,0.015,0.055,0.279,0.157,0.232,0.083,0.057,0.520
17518,2013-12-31,2013-12-31 23:00:00,0.491,0.027,0.066,0.142,0.124,0.172,0.110,0.033,0.578


In [31]:
h2

,DateTime,h9
0,2013-07-19 00:30:00,0.086
1,2013-07-19 01:00:00,0.117
2,2013-07-19 01:30:00,0.086
3,2013-07-19 02:00:00,0.128
4,2013-07-19 02:30:00,0.076
...,...,...
39172,2013-07-18 22:30:00,0.053
39173,2013-07-18 23:00:00,0.036
39174,2013-07-18 23:30:00,0.079
39175,2013-07-19 00:00:00,0.050


In [22]:
smart_meter_data

,DateTime,KWH/hh (per half hour),house,Date,h2
0,2013-01-01 00:00:00,0.219,1,2013-01-01,NaN
1,2013-01-01 00:30:00,0.241,1,2013-01-01,NaN
2,2013-01-01 01:00:00,0.191,1,2013-01-01,NaN
3,2013-01-01 01:30:00,0.235,1,2013-01-01,NaN
4,2013-01-01 02:00:00,0.182,1,2013-01-01,NaN
...,...,...,...,...,...
17515,2013-12-31 21:30:00,0.451,1,2013-12-31,0.094
17516,2013-12-31 22:00:00,0.456,1,2013-12-31,0.126
17517,2013-12-31 22:30:00,0.402,1,2013-12-31,0.124
17518,2013-12-31 23:00:00,0.491,1,2013-12-31,0.105


In [ ]:
smart_meter_data.dtypes

In [ ]:
sdate  = np.min(smart_meter_data['Date'])
edate  = np.max(smart_meter_data['Date'])
dts = list(set(pd.date_range(sdate,edate-timedelta(days=1),freq='d')))
print(sdate, edate, len(dts))

In [ ]:
smart_meter_data.head(20)

In [ ]:
# check all days have 48 (HH) reads
date_counts = smart_meter_data['Date'].value_counts()
#date_counts.plot(kind='bar')
date_counts.plot()

In [ ]:
min_pwr  = np.min(smart_meter_data['KWH/hh (per half hour) '])
max_pwr  = np.max(smart_meter_data['KWH/hh (per half hour) '])

print(min_pwr, max_pwr)

In [ ]:
smart_meter_data.head(10)

In [ ]:
day = smart_meter_data[smart_meter_data['Date'] == '2013-09-17']
day.shape

In [ ]:
day.head(len(day))

In [ ]:
# show the days profile
print('max = ', np.min(day['KWH/hh (per half hour) ']))
print('min = ', np.max(day['KWH/hh (per half hour) ']))

plt.plot(day['DateTime'], day['KWH/hh (per half hour) '])


#yticks = np.arange(0, 10, 2)

#plt.ylabel('house 1 demand')
plt.ylabel('Demand kW')
plt.xlabel('Time of day')
plt.xticks(rotation = 45)
plt.show()

In [ ]:
#smart_meter_data.to_csv(dirpath+'/all meters.csv')

In [ ]:
# weather
weather_data = pd.read_csv(dirpath+'/GBR_London.Gatwick.037760_IWEC_copy.epw', sep='\t', 
                           usecols=['year', 'month', 'day', 'hour', 'minute', 
                                    'Global Horizontal Radiation', 'Total Sky Cover'],
                                    low_memory=False)

weather_data.head(10)

In [ ]:
# weather data is hourly but smart meter data is half-hourly
# simply duplicate to half-hourly
modified_weather_data = pd.DataFrame(columns=['DateTime', 'Date', 'Global Horizontal Radiation', 'Total Sky Cover'])

for i in trange(len(weather_data)):
    
    # ignore the weather data year and match to demand file which is 2013
    new_date = '2013-' + str(weather_data.loc[i,'month']) + '-' + str(weather_data.loc[i,'day'])
    
    # weather data uses hours 1 to 24, we want 0-23, and produce on-hour and half-hour timings
    on_hr = new_date + ' ' + str(weather_data.loc[i,'hour']-1) + ':00:00'
    on_hh = new_date + ' ' + str(weather_data.loc[i,'hour']-1) + ':30:00'
    
    # convert to datetimes
    on_hr = datetime.strptime(on_hr, '%Y-%m-%d %H:%M:%S')
    on_hh = datetime.strptime(on_hh, '%Y-%m-%d %H:%M:%S')
    new_date = datetime.strptime(new_date, '%Y-%m-%d')
    
    ghr = weather_data.loc[i,'Global Horizontal Radiation'] # Wh/m2
    tsc = weather_data.loc[i,'Total Sky Cover'] # 10ths of sky cover, e.g. 2 => 2/10ths cloud
    
    modified_weather_data = modified_weather_data.append({'DateTime': on_hr, 'Date': new_date, 'Global Horizontal Radiation': ghr, 'Total Sky Cover': tsc}, ignore_index=True)
    modified_weather_data = modified_weather_data.append({'DateTime': on_hh, 'Date': new_date, 'Global Horizontal Radiation': ghr, 'Total Sky Cover': tsc}, ignore_index=True)
    

In [ ]:
modified_weather_data.head(30)

In [ ]:
modified_weather_data['Global Horizontal Radiation'].value_counts()

Solar PV class

In [ ]:
from IPython.display import Image
dirpath = r'C:\Users\joeba\OneDrive\Documents\ML\smart grid'
Image(dirpath + '/solar-pv.jpg') #, width=1000, height=800)

In [ ]:
class SolarPV():
    
    def __init__(self, kWp, efficiency, performance_ratio):
        self.kWp = kWp
        self.efficiency = efficiency
        self.performance_ratio = performance_ratio
        
    ### sort these functions out when get back to Hayling and have CAT notes
    def output(self, ghi):
        # ghi is in Wh/m2, so divide by 1000 to get kWh
        op = 0.001 * ghi * self.kWp * self.efficiency * self.performance_ratio
        return np.round(op, 3)
    
    ### sort these functions out when get back to Hayling and have CAT notes
    def output2(self, ghi, tsc):
        # account for cloudy sky
        op = 0.001 * ghi * self.kWp * self.efficiency * self.performance_ratio * (1 -(tsc/10))
        return np.round(op, 3)

In [ ]:
pv = SolarPV(4.24, 0.3, 0.8)

generated = []

for i in trange(len(weather_data)):
    generated.append(pv.output(modified_weather_data.loc[i, 'Global Horizontal Radiation']))
    #generated.append(pv.output(modified_weather_data.loc[i, 'Global Horizontal Radiation'], modified_weather_data.loc[i, 'Total Sky Cover']))

In [ ]:
len(generated)

In [ ]:
generated

In [ ]:
np.max(generated)

In [ ]:
pv.output(1828)